In [1]:
import pandas as pd
import numpy as np
# 绘图相关的库
import os
import matplotlib.pyplot as plt
from utils import single_point_positioning, ecef_to_llh, llh_to_enu, plot_trajectory, weighted_single_point_positioning
from eval_utils import multi_constellation_spp, weighted_multi_constellation_spp

✅ 最终版5参数混合解算器 (GNSS + PL) 已准备就绪。


In [2]:
dir_path = os.getcwd()
#lstm_pred_data ='learning/real_with_predicted_multipath_LSTM.csv'# this is the result from lstm model
real_world_data ='RWD_20250422/D1T4/observation_data_case5.csv'
svm_pred_data ='learning/real_with_predicted_multipath_svm_bal.csv'
uad_svm_pred_data ='RWD_20250422/D1T4/tgt_data_with_single_svm_predictions.csv'
reference_truth_data ='RWD_20250422/D1T4/user_pos_full.csv'
yuanlai_data ='RWD_20250422/D1T4/UAD_SVM_observation_data_case5_WITH_PREDICTIONS.csv'
#lstm_pred = pd.read_csv(os.path.join(dir_path, lstm_pred_data))
#svm_pred = pd.read_csv(os.path.join(dir_path, svm_pred_data))
yuanlai = pd.read_csv(os.path.join(dir_path, yuanlai_data))
real_world = pd.read_csv(os.path.join(dir_path, real_world_data))
uad_svm_pred = pd.read_csv(os.path.join(dir_path, uad_svm_pred_data))
svm_pred = pd.read_csv(os.path.join(dir_path, svm_pred_data))
reference_truth_data = pd.read_csv(os.path.join(dir_path, reference_truth_data))
#print(f"正在从 '{lstm_pred_data}' 加载观测数据...")
#print(f"正在从 '{svm_pred_data}' 加载观测数据...")
#print(f"正在从 '{uad_svm_pred_data}' 加载观测数据...")
print(f"正在从 '{yuanlai_data}' 加载观测数据...")
#display(lstm_pred.head())
#display(svm_pred.head())
display(uad_svm_pred.head())



#真实地面值的坐标转换计算
ecef_coords = reference_truth_data[['x', 'y', 'z']].values
llh_coords = ecef_to_llh(ecef_coords)
reference_truth_data['lat'] = llh_coords[:, 0]
reference_truth_data['lon'] = llh_coords[:, 1]
reference_truth_data['h'] = llh_coords[:, 2]

正在从 'RWD_20250422/D1T4/UAD_SVM_observation_data_case5_WITH_PREDICTIONS.csv' 加载观测数据...


,gps_time,satellite_id,sv_id,pseudorange,doppler_shift,cn0,azimuth,elevation,pseudorange_residual,pseudorange_corrected_cb,sat_px,sat_py,sat_pz,sat_type,predicted_multipath
0,1.429365e+09,G01,1,2.371945e+07,-3039.00801,42.97546,262.63392,15.78166,-0.46956,2.409778e+07,1.527411e+07,-2.033628e+07,7.613143e+06,0,NaN
1,1.429365e+09,G02,2,2.160127e+07,-1837.17348,45.60996,277.05302,44.45788,0.04192,2.185358e+07,1.592333e+07,-1.252732e+07,1.739108e+07,0,0.0
2,1.429365e+09,G08,8,2.007511e+07,-926.01404,48.38570,290.42208,66.98577,-0.61199,2.053719e+07,1.602714e+07,-4.383432e+06,2.065940e+07,0,NaN
3,1.429365e+09,G10,10,2.069873e+07,849.49657,48.68555,76.04460,65.66533,0.83612,2.088786e+07,1.343987e+07,1.129458e+07,2.027075e+07,0,NaN
4,1.429365e+09,G23,23,2.277589e+07,3131.67460,39.14048,47.37784,25.36873,-4.55863,2.322375e+07,-2.775320e+06,1.519344e+07,2.162021e+07,0,1.0


In [3]:
grouped_data = uad_svm_pred.groupby('gps_time')

#grouped_data = yuanlai.groupby('gps_time')
#num_unique_times = svm_pred['gps_time'].nunique()
#num_unique_times_real_world= real_world['gps_time'].nunique()
#print(f"真实观测数据总共有: {num_unique_times_real_world} 个不同的（不重复的）时间点。")
num_unique_times_yuanlai= yuanlai['gps_time'].nunique()
print(f" 总共有: {num_unique_times_yuanlai} 个不同的（不重复的）时间点。")

results_spp = {}        # 情况1: 标准SPP (5参数)
results_wls = {}        # 情况2: 加权SPP (5参数, WLS)
results_exclusion = {}  # 情况3: 剔除坏信号后的SPP (5参数)

print("开始处理每个时间点的数据...")
print(grouped_data.size())
print(f"总共有 {len(grouped_data)} 个独立的时间点。")


 总共有: 84 个不同的（不重复的）时间点。
开始处理每个时间点的数据...
gps_time
1.429365e+09     5
1.429365e+09     5
1.429365e+09     5
1.429365e+09     5
1.429365e+09     5
                ..
1.429365e+09    17
1.429365e+09    13
1.429365e+09    17
1.429365e+09    17
1.429365e+09    17
Length: 832, dtype: int64
总共有 832 个独立的时间点。


In [4]:
for timestamp, group in grouped_data:
    
    # ------------------ [修正 1] ------------------
    # 解算 (X,Y,Z, dt_gps, isb_pl) 至少需要 5 颗卫星
    if len(group) < 5: 
        continue
    # ----------------------------------------------
    unique_sat_types = group['sat_type'].unique()
    if len(unique_sat_types) < 2:
        # 如果这个时间点只有GPS(0)，或只有PL(1)，解算一定会失败，必须跳过
        continue    
    # 提取所有信号的数据
    all_sat_positions = group[['sat_px', 'sat_py', 'sat_pz']].values
    all_pseudoranges = group['pseudorange_corrected_cb'].values
    # ------------------ [修正 2] ------------------
    # 必须提取 sat_type 以区分 GPS / PL
    all_sat_types = group['sat_type'].values 
   
    # ----------------------------------------------

    # --- 情况一：标准SPP ---
    try:
        spp_pos, _, _ = multi_constellation_spp(all_sat_positions, all_pseudoranges, all_sat_types)
        results_spp[timestamp] = spp_pos
    except Exception as e:
        pass # 解算失败，跳过

    # --- 情况二：加权SPP (WLS) (您的SVM模型应用在这里) ---
    try:
        weights = group['predicted_multipath'].apply(lambda x: 0.1 if x == 1 else 1.0).values
        wls_pos, _, _ = weighted_multi_constellation_spp(all_sat_positions, all_pseudoranges, all_sat_types, weights)
        results_wls[timestamp] = wls_pos
    except Exception as e:
        pass # 解算失败，跳过

    # --- 情况三：剔除坏信号SPP ---
    good_signals_df = group[group['predicted_multipath'] == 0]
    
    # 同样，对剔除后的数据也要进行这两个检查
    good_unique_types = good_signals_df['sat_type'].unique()
    
    if len(good_signals_df) >= 5 and len(good_unique_types) >= 2:
        try:
            good_sat_positions = good_signals_df[['sat_px', 'sat_py', 'sat_pz']].values
            good_pseudoranges = good_signals_df['pseudorange_corrected_cb'].values
            good_sat_types = good_signals_df['sat_type'].values
            
            exclusion_pos, _, _ = multi_constellation_spp(good_sat_positions, good_pseudoranges, good_sat_types)
            results_exclusion[timestamp] = exclusion_pos
        except Exception as e:
            pass # 解算失败，跳过
            
print("所有数据处理完毕")
print(f"标准SPP: {len(results_spp)} 个点")
print(f"加权WLS: {len(results_wls)} 个点")
print(f"剔除坏信号SPP: {len(results_exclusion)} 个点")

所有数据处理完毕
标准SPP: 697 个点
加权WLS: 697 个点
剔除坏信号SPP: 693 个点


In [5]:

# 定义ENU坐标系的原点 (使用参考轨迹的第一个点)
origin_llh = reference_truth_data[['lat', 'lon', 'h']].iloc[0].values

# 将参考轨迹也转换为ENU以进行对比
reference_llh = reference_truth_data[['lat', 'lon', 'h']].values
reference_enu = llh_to_enu(reference_llh, origin_llh)

# 存储所有ECEF结果，方便循环处理
all_results_ecef = {
    '1. SPP': results_spp,
    '2. WLS ': results_wls,
    '3. Exclusion SPP': results_exclusion
}

all_results_enu = {}
for name, ecef_results in all_results_ecef.items():
    if ecef_results:
        ecef_points = np.array(list(ecef_results.values()))
        llh_points = ecef_to_llh(ecef_points)
        enu_trajectory = llh_to_enu(llh_points, origin_llh)
        all_results_enu[name] = enu_trajectory
        print(f"{name} 的ECEF坐标已转换为ENU坐标，包含 {len(enu_trajectory)} 个点。")
print("所有场景的ECEF坐标已成功转换为ENU坐标。")

1. SPP 的ECEF坐标已转换为ENU坐标，包含 697 个点。
2. WLS  的ECEF坐标已转换为ENU坐标，包含 697 个点。
3. Exclusion SPP 的ECEF坐标已转换为ENU坐标，包含 693 个点。
所有场景的ECEF坐标已成功转换为ENU坐标。


In [6]:
%matplotlib inline
%matplotlib qt
# 准备最终的绘图数据字典
plot_data = {
    "Ground Truth": reference_enu,#来自user_pos_full.csv的参考轨迹
    **all_results_enu  # 使用字典解包，将所有计算出的轨迹加入
}

# 调用您自己编写的绘图函数，并设置 scatter=True
fig = plot_trajectory(
    plot_data,
    title=" (ENU coordinate)",
    type="enu",
    twoD=True,
    scatter=True 
)

# 添加更多绘图细节，让图像更清晰
plt.xlabel("East (m)")
plt.ylabel("North (m)")
plt.grid(True)
plt.axis('equal') 
plt.legend()
plt.show()

In [7]:
%matplotlib qt
# 准备最终的绘图数据字典
plot_data = {
    "(Ground Truth)": reference_enu,
    **all_results_enu
}

# 调用您自己编写的绘图函数

fig = plot_trajectory(
    plot_data,
    title="(ENU coordinate)",
    type="enu",
    twoD=True,
    scatter=False
)

# 您可以在此基础上添加更多绘图细节
plt.xlabel("East (m)")
plt.ylabel("North (m)")
# plt.axis('equal') # 保持XY轴比例一致，如果轨迹范围太大可能导致图像被压缩
plt.axis('equal')
plt.legend()
plt.show()

In [8]:
# 1️⃣ 准备真值 (ref_df)
ref_df = reference_truth_data[['gps_time', 'x', 'y', 'z']].copy()
# 必须将 gps_time 保持为 float (浮点数) 类型
ref_df['gps_time'] = ref_df['gps_time'].astype(float) 
ref_df = ref_df.sort_values('gps_time').reset_index(drop=True)

# 2️⃣ 准备估计结果 (est_df) - 以 "加权WLS" (情况2) 为例
# (您应该为您关心的那组结果计算误差)
est_df = pd.DataFrame([
    [t, pos[0], pos[1], pos[2]] for t, pos in results_wls.items()
], columns=['gps_time', 'x_est', 'y_est', 'z_est'])

# 确保 est_df 的时间也是 float
if not est_df.empty:
    est_df['gps_time'] = est_df['gps_time'].astype(float)
    est_df = est_df.sort_values('gps_time').reset_index(drop=True)
else:
    print("错误：估计结果 (est_df) 为空，无法计算误差。")
    # 在这里可以停止或跳过

print(f"真实轨迹: {len(ref_df)} 点, 估计结果: {len(est_df)} 点")

# 3️⃣ 使用 pandas.merge_asof 按时间对齐
# 两个DataFrame的 'gps_time' 都是 float 了，
# 'tolerance=0.5' (float) 才能正常工作
aligned = pd.merge_asof(
    est_df, ref_df,
    on='gps_time',
    direction='nearest', # 寻找最近的时间戳
    tolerance=0.5      # 允许的最大时差 (0.5秒)
).dropna()

print(f"成功匹配的点数: {len(aligned)}")

# 4️⃣ 坐标转换：ECEF → ENU
if not aligned.empty:
    origin_llh = ecef_to_llh(ref_df[['x', 'y', 'z']].values[:1])[0]
    ref_llh = ecef_to_llh(aligned[['x', 'y', 'z']].values)
    est_llh = ecef_to_llh(aligned[['x_est', 'y_est', 'z_est']].values)
    
    ref_enu = llh_to_enu(ref_llh, origin_llh)
    est_enu = llh_to_enu(est_llh, origin_llh)

# 5️⃣ 计算误差
errors = est_enu - ref_enu
mean_error = np.mean(errors, axis=0)
rmse = np.sqrt(np.mean(errors**2, axis=0))
rmse_2d = np.sqrt(rmse[0]**2 + rmse[1]**2)
rmse_3d = np.sqrt(np.sum(rmse**2))

# 6️⃣ 打印结果
print("\n=== 剔除坏信号 (Good Signals Only) 性能评估 ===")
print(f"样本数: {len(aligned)} / 匹配成功率: {len(aligned)/len(est_df)*100:.1f}%")
print(f"Mean 误差 (E, N, U) [m]: {mean_error}")
print(f"RMSE 误差 (E, N, U) [m]: {rmse}")
#print(f"2D RMSE: {rmse_2d:.3f} m | 3D RMSE: {rmse_3d:.3f} m")

# 7️⃣ 可视化误差随时间
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(errors[:,0], label='East Error')
plt.plot(errors[:,1], label='North Error')
plt.plot(errors[:,2], label='Up Error')
plt.xlabel('Epoch Index')
plt.ylabel('Error (m)')
plt.legend()
plt.grid(True)
plt.title('ENU Error over Time (Good Signal SPP)')
plt.show()

真实轨迹: 1111 点, 估计结果: 697 点
成功匹配的点数: 697

=== 剔除坏信号 (Good Signals Only) 性能评估 ===
样本数: 697 / 匹配成功率: 100.0%
Mean 误差 (E, N, U) [m]: [1.2726297  2.18387597 0.27182652]
RMSE 误差 (E, N, U) [m]: [ 9.17279588 15.04599764  4.38919901]
